In [1]:
import pandas as pd
import gensim
from gensim.parsing.preprocessing import preprocess_documents
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# References:
- https://rasa.com/docs/rasa/testing-your-assistant/


In [2]:
ls

1a_Get_AirbnbData.ipynb
1b_quora-scrapper[inActive].ipynb
1c_Twitter-read[inActive].ipynb
1d_Twitter_APIKey.ipynb
2a_SentimentAnalyzer.ipynb
2b_FAQ_Questgen_NLP.ipynb
2c_ProcessText.ipynb
2d_Neo4jData_Gen.ipynb
3a_EmbeddingsAndModels.ipynb
4a_GenerateRASA_Conversational_Data.ipynb
5a_offlineRecommendation.ipynb
5b_realTimeRecommendation.ipynb
6a_ConceptNet5Query.ipynb
Chatbot-Widget/
Chatbot-Widget.zip
Data/
Documents/
LICENSE
Papers/
Personas/
Preprocessing.ipynb
RASA/
RASA_ConceptNet5/
README.md
mem_cache_conceptNet5.json
plugins/
s2v_old/
s2v_reddit_2015_md.tar.gz
s2v_reddit_2015_md.tar.gz.1
s2v_reddit_2015_md.tar.gz.2
s2v_reddit_2015_md.tar.gz.3
s2v_reddit_2015_md.tar.gz.4
s2v_reddit_2015_md.tar.gz.5
s2v_reddit_2015_md.tar.gz.6


In [3]:
root = './RASA/data/'
model_root='./RASA/offline_models/'
processed = './Data/processing/Processed_Airbnb/'
raw = './Data/raw/'

listings = pd.read_csv(raw+'listings.csv.gz', sep=',')
reviews = pd.read_csv(processed+'ratings_filter.csv', sep=',')
listings=listings.rename(columns={"id": "listing_id"})
listings.to_csv(processed+'listings.csv.gz', sep=',')

In [4]:
# drop columns that have no value to our recommendation
listings = listings[['listing_id','listing_url','name','description','neighborhood_overview','picture_url', 
'property_type','room_type','accommodates','bathrooms','bathrooms_text',                               
'bedrooms','beds','amenities','price','minimum_nights','maximum_nights','review_scores_rating',                         
'review_scores_accuracy','review_scores_cleanliness','review_scores_checkin',
'review_scores_communication','review_scores_location']]

listings.fillna('0', inplace=True)

listings.reset_index(drop = True, inplace = True)

In [5]:
import string
def remove_punc(sample_str):
    # Create translation table in which special charcters
    # are mapped to empty string
    translation_table = str.maketrans('', '', string.punctuation)
    # Remove special characters from the string using translation table
    sample_str = sample_str.translate(translation_table)
    return sample_str

In [6]:
listings['words_features'] = listings['amenities'].apply(remove_punc)

for ind in listings.index:
     listings['review_scores_rating'][ind] = (float(listings['review_scores_rating'][ind]) + float(listings['review_scores_accuracy'][ind]) + float(listings['review_scores_cleanliness'][ind]) + float(listings['review_scores_checkin'][ind]) + float(listings['review_scores_communication'][ind]) + float(listings['review_scores_location'][ind]))
     listings['review_scores_rating'][ind] = (listings['review_scores_rating'][ind])/6
     listings['words_features'][ind] = 'amenities:'+listings['words_features'][ind] +'description:'+  listings['description'][ind] +'neighborhood_overview:'+  listings['neighborhood_overview'][ind]+'property_type:'+  listings['property_type'][ind]+'room_type:'+  listings['room_type'][ind]+'accommodates:'+  str(listings['accommodates'][ind])+'bedrooms:'+  str(listings['bedrooms'][ind])+'beds:'+  str(listings['beds'][ind])+'price range:'+  listings['price'][ind]
listingss = listings.rename(columns={"review_scores_rating": "overall_rating"})

/var/folders/s8/wjstlrhj169345phv980cwx80000gn/T/ipykernel_14475/2060811252.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings['review_scores_rating'][ind] = (float(listings['review_scores_rating'][ind]) + float(listings['review_scores_accuracy'][ind]) + float(listings['review_scores_cleanliness'][ind]) + float(listings['review_scores_checkin'][ind]) + float(listings['review_scores_communication'][ind]) + float(listings['review_scores_location'][ind]))
/var/folders/s8/wjstlrhj169345phv980cwx80000gn/T/ipykernel_14475/2060811252.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings['review_scores_rating'

In [7]:
text_corpus = listingss['words_features'].values

In [8]:
processed_corpus = preprocess_documents(text_corpus)
tagged_corpus = [TaggedDocument(d, [i]) for i, d in enumerate(processed_corpus)]

In [9]:
model = Doc2Vec(tagged_corpus, dm=0, vector_size=200, window=2, min_count=1, epochs=100, hs=1)

In [11]:
model1= model
model.save('RASA/data/embeddings/lst_embeddings')

In [12]:
new_doc = gensim.parsing.preprocessing.preprocess_string("private room dishwasher")
test_doc_vector = model1.infer_vector(new_doc)
sims = model.docvecs.most_similar(positive = [test_doc_vector])
for s in sims:
    print(f"{(s[1])} | {listings['listing_url'].iloc[s[0]]}")

0.452333927154541 | https://www.airbnb.com/rooms/25991654
0.43859487771987915 | https://www.airbnb.com/rooms/16685383
0.4372003972530365 | https://www.airbnb.com/rooms/49927889
0.4354342520236969 | https://www.airbnb.com/rooms/51961840
0.430204302072525 | https://www.airbnb.com/rooms/8641456
0.4221857786178589 | https://www.airbnb.com/rooms/40247038
0.42049500346183777 | https://www.airbnb.com/rooms/32729549
0.4142203629016876 | https://www.airbnb.com/rooms/39642322
0.40605443716049194 | https://www.airbnb.com/rooms/6356312
0.39622607827186584 | https://www.airbnb.com/rooms/16737594


/var/folders/s8/wjstlrhj169345phv980cwx80000gn/T/ipykernel_14475/2108765969.py:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims = model.docvecs.most_similar(positive = [test_doc_vector])
